In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nasadata/data/mars/training/data/XB.ELYSE.02.BHV.2022-01-02HR04_evid0006.mseed
/kaggle/input/nasadata/data/mars/training/data/XB.ELYSE.02.BHV.2022-01-02HR04_evid0006.csv
/kaggle/input/nasadata/data/mars/training/data/XB.ELYSE.02.BHV.2022-02-03HR08_evid0005.mseed
/kaggle/input/nasadata/data/mars/training/data/XB.ELYSE.02.BHV.2022-02-03HR08_evid0005.csv
/kaggle/input/nasadata/data/mars/training/catalogs/Mars_InSight_training_catalog_final.csv
/kaggle/input/nasadata/data/mars/training/catalogs/Mars_InSight_training_catalog.csv
/kaggle/input/nasadata/data/mars/training/plots/evid_0006.png
/kaggle/input/nasadata/data/mars/training/plots/evid_0005.png
/kaggle/input/nasadata/data/mars/test/data/XB.ELYSE.02.BHV.2022-05-04HR23_evid0001.mseed
/kaggle/input/nasadata/data/mars/test/data/XB.ELYSE.02.BHV.2019-07-26HR12_evid0034.mseed
/kaggle/input/nasadata/data/mars/test/data/XB.ELYSE.02.BHV.2019-05-23HR02_evid0041.mseed
/kaggle/input/nasadata/data/mars/test/data/XB.ELYSE.02.BHV.2019-0

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout , BatchNormalization,Input
from keras.callbacks import EarlyStopping , ModelCheckpoint
from scipy.signal import butter, lfilter


In [3]:
import tensorflow as tf
tf.__version__

'2.16.1'

In [4]:
import os

# # Set the directory for your training data
train_data_dir = '/kaggle/input/nasadata/data/lunar/training/data/S12_GradeA'

# # List to hold DataFrames
dataframes = []

# # Loop through all CSV files in the training folder
for filename in os.listdir(train_data_dir):
    if filename.endswith('.csv'):
        file_path = os.path.join(train_data_dir, filename)
        df = pd.read_csv(file_path)
        dataframes.append(df)

# # Concatenate all DataFrames into a single DataFrame
training_data = pd.concat(dataframes, ignore_index=True)
# training_data = pd.read_csv('C:/Users/nm_ma/Desktop/space_apps_2024_seismic_detection/data/lunar/training/data/S12_GradeA/xa.s12.00.mhz.1970-01-19HR00_evid00002.csv')
# Display the first few rows of the merged DataFrame
print(training_data.head())


  time_abs(%Y-%m-%dT%H:%M:%S.%f)  time_rel(sec)  velocity(m/s)
0     1974-07-06T00:00:00.193000       0.000000  -5.822985e-16
1     1974-07-06T00:00:00.343943       0.150943  -6.922769e-16
2     1974-07-06T00:00:00.494887       0.301887  -7.255003e-16
3     1974-07-06T00:00:00.645830       0.452830  -6.945061e-16
4     1974-07-06T00:00:00.796774       0.603774  -6.315960e-16


In [5]:
training_data.tail()

,time_abs(%Y-%m-%dT%H:%M:%S.%f),time_rel(sec),velocity(m/s)
43175424,1974-04-19T23:59:57.700075,86397.132075,1.787332e-16
43175425,1974-04-19T23:59:57.851019,86397.283019,-1.354541e-15
43175426,1974-04-19T23:59:58.001962,86397.433962,-1.946218e-15
43175427,1974-04-19T23:59:58.152906,86397.584906,-2.319162e-15
43175428,1974-04-19T23:59:58.303849,86397.735849,-3.208193e-15


In [6]:
import tensorflow as tf

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  2


In [7]:
import pandas as pd

def label_matching_rows(training_data_df, catalog_file):
    # Read the catalog file
    catalog_df = pd.read_csv(catalog_file)

    # Ensure the timestamp in the catalog is in the correct format
    catalog_df['time_abs'] = pd.to_datetime(catalog_df['time_abs(%Y-%m-%dT%H:%M:%S.%f)'], format='%Y-%m-%dT%H:%M:%S.%f').dt.floor('D')

    # Convert time_abs in the training DataFrame to datetime format
    training_data_df['time_abs'] = pd.to_datetime(training_data_df['time_abs(%Y-%m-%dT%H:%M:%S.%f)'], format='%Y-%m-%dT%H:%M:%S.%f').dt.floor('D')

    # Initialize target column with 0
    training_data_df['target'] = 0

    # Get unique catalog days
    unique_catalog_days = catalog_df['time_abs'].unique()

    # Set target to 1 for the first occurrence of matching days in the training data
    for catalog_day in unique_catalog_days:
        # Find the first occurrence of this catalog_day in the training data
        first_occurrence_index = training_data_df[training_data_df['time_abs'] == catalog_day].index.min()
        if first_occurrence_index is not None:
            training_data_df.loc[first_occurrence_index, 'target'] = 1

    # Drop unnecessary columns, keep only relevant ones
    columns_to_keep = ['time_abs', 'time_rel(sec)', 'velocity(m/s)', 'target']
    training_data_df = training_data_df[columns_to_keep]

    # Drop duplicate entries if necessary
    training_data_df.drop_duplicates(inplace=True)

    return training_data_df

# Labeling using the catalog file
catalog_file = '/kaggle/input/nasadata/data/lunar/training/catalogs/apollo12_catalog_GradeA_final.csv'
labeled_training_data = label_matching_rows(training_data, catalog_file)

# Display the first few rows of the labeled DataFrame
print("Labeled Training Data:")
print(labeled_training_data.head())

# Check counts of target values
print("Target variable counts:")
print(labeled_training_data['target'].value_counts())


Labeled Training Data:
    time_abs  time_rel(sec)  velocity(m/s)  target
0 1974-07-06       0.000000  -5.822985e-16       1
1 1974-07-06       0.150943  -6.922769e-16       0
2 1974-07-06       0.301887  -7.255003e-16       0
3 1974-07-06       0.452830  -6.945061e-16       0
4 1974-07-06       0.603774  -6.315960e-16       0
Target variable counts:
target
0    40313341
1          71
Name: count, dtype: int64


/tmp/ipykernel_55/2947508011.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  training_data_df.drop_duplicates(inplace=True)


In [8]:
labeled_training_data.tail()

,time_abs,time_rel(sec),velocity(m/s),target
43175424,1974-04-19,86397.132075,1.787332e-16,0
43175425,1974-04-19,86397.283019,-1.354541e-15,0
43175426,1974-04-19,86397.433962,-1.946218e-15,0
43175427,1974-04-19,86397.584906,-2.319162e-15,0
43175428,1974-04-19,86397.735849,-3.208193e-15,0


In [9]:
labeled_training_data['target'].value_counts()

target
0    40313341
1          71
Name: count, dtype: int64

In [10]:
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a

def bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y


In [11]:
# Set filter parameters
lowcut = 0.1  # Set low cutoff frequency
highcut = 10.0  # Set high cutoff frequency
fs = 100.0  # Sample rate (Hz)

# Apply filter to the velocity data
labeled_training_data.loc[:, 'filtered_velocity'] = bandpass_filter(labeled_training_data['velocity(m/s)'].values, lowcut, highcut, fs)


/tmp/ipykernel_55/2635595105.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  labeled_training_data.loc[:, 'filtered_velocity'] = bandpass_filter(labeled_training_data['velocity(m/s)'].values, lowcut, highcut, fs)


In [12]:
labeled_training_data

,time_abs,time_rel(sec),velocity(m/s),target,filtered_velocity
0,1974-07-06,0.000000,-5.822985e-16,1,-7.155067e-19
1,1974-07-06,0.150943,-6.922769e-16,0,-6.559821e-18
2,1974-07-06,0.301887,-7.255003e-16,0,-2.910505e-17
3,1974-07-06,0.452830,-6.945061e-16,0,-8.442040e-17
4,1974-07-06,0.603774,-6.315960e-16,0,-1.823950e-16
...,...,...,...,...,...
43175424,1974-04-19,86397.132075,1.787332e-16,0,5.904726e-15
43175425,1974-04-19,86397.283019,-1.354541e-15,0,5.521393e-15
43175426,1974-04-19,86397.433962,-1.946218e-15,0,4.914140e-15
43175427,1974-04-19,86397.584906,-2.319162e-15,0,3.993375e-15


In [13]:
scaler = MinMaxScaler(feature_range=(0, 1))
labeled_training_data.loc[:, 'filtered_velocity'] = scaler.fit_transform(labeled_training_data[['filtered_velocity']])


In [14]:
def create_sequences(data, target, time_steps):
    X, y = [], []
    for i in range(len(data) - time_steps):
        X.append(data[i:(i + time_steps)])  # Feature sequences
        y.append(target[i + time_steps])     # Corresponding target
    return np.array(X), np.array(y)

# Set time steps
time_steps = 10  # Number of time steps to look back

# Create sequences
X, y = create_sequences(labeled_training_data['filtered_velocity'].values, labeled_training_data['target'].values, time_steps)

# Reshape X for LSTM input: (samples, time steps, features)
X = X.reshape((X.shape[0], X.shape[1], 1))  # 1 feature in this case

# Display the shapes of the resulting arrays
print("Shape of X:", X.shape)  # Should be (num_samples, time_steps, num_features)
print("Shape of y:", y.shape)  # Should be (num_samples,)

Shape of X: (40313402, 10, 1)
Shape of y: (40313402,)


In [15]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [17]:
# Define the LSTM model
model = Sequential()

# Input layer
model.add(Input(shape=(X_train.shape[1], X_train.shape[2])))  # Specify the input shape

# First LSTM layer
model.add(LSTM(64, return_sequences=True))  # Set return_sequences=True for stacking LSTMs
model.add(BatchNormalization())
model.add(Dropout(0.3))  # Dropout to prevent overfitting

# Second LSTM layer
model.add(LSTM(32))  # No need for return_sequences=True on the last LSTM
model.add(BatchNormalization())
model.add(Dropout(0.3))  # Dropout to prevent overfitting

# Output layer for binary classification
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Early stopping callback to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

# Train the model
history = model.fit(X_train, y_train,
                    validation_data=(X_val, y_val),
                    epochs=1,  # Training for 5 epochs
                    batch_size=64,
                    callbacks=[early_stopping],
                    verbose=1)  # Set verbose to 1 for progress output

503918/503918 ━━━━━━━━━━━━━━━━━━━━ 3967s 8ms/step - accuracy: 0.9994 - loss: 0.0020 - val_accuracy: 1.0000 - val_loss: 6.1606e-05


In [18]:
# Save the model to a file
model.save('lunarmodel.keras')